In [ ]:
from completion_executor import ChatCompletionExecutor
from sliding_window_executor import SlidingWindowExecutor
import json

In [ ]:
with open("news_info.json", "r") as f:
    news_info = json.load(f)

In [ ]:
instruction = "다음의 과제를 수행해줘.\r\n\r\n-목표-\r\n기업 관련 뉴스 기사와 객체의 유형들이 제시되어 있는 목록이 주어졌을 때, 목록에 포함된 유형에 해당하는 객체와 식별된 객체 간의 관계를 모두 파악하기.\r\n\r\n-단계-\r\n1. 모든 객체를 식별하기. 식별된 객체 별로 다음의 정보를 추출하기:\r\n- 객체_이름: 객체의 이름.\r\n- 객체_유형: 객체 유형 목록에서 식별된 객체와 가장 적합한 유형 하나를 선택.\r\n- 객체_설명: 객체의 특성과 활동에 대한 종합적인 설명.\r\n(\"객체\",<객체_이름>,<객체_유형>,<객체_설명>)의 형식으로 객체를 구성.\r\n\r\n2. 1단계에서 얻은 객체들을 활용하여 서로 명확하게 연관되어있는 모든 객체 쌍 (객체_1, 객체_2)를 식별하기.\r\n각 연관된 객체 쌍 별로 다음의 정보를 추출하기:\r\n- 객체_1: 1단계에서 식별된 객체의 이름.\r\n- 객체_2: 1단계에서 식별된 객체의 이름.\r\n- 관계_설명: 객체_1과 객체_2가 서로 연관되어있다고 생각하는 이유에 대한 설명.\r\n- 관계_강도: 객체_1과 객체_2 사이의 관계가 각 객체와 연관된 기업의 주가 관점에서 긍정적인지, 또는 부정적인지를 나타내는 점수로 -5에서 +5의 범위를 가지며 양수는 객체_1과 객체_2의 주가(혹은 객체_1과 객체_2와 연관된 기업의 주가)에 있어서 긍정적인 관계, 음수는 부정적인 관계를 나타낸다. 관계 강도의 절대값이 클수록 영향력이 강한 관계를 의미한다.\r\n- 관계_날짜: 뉴스 기사 작성일.\r\n(\"관계\",<객체_1>,<객체_2>,<관계_설명>,<관계_강도>, <관계_날짜>)의 형식으로 객체를 구성.\r\n\r\n3. 1단계와 2단계에서 얻은 모든 객체와 관계의 정보를 출력하기.\r\n\r\n위 과제의 예시는 다음과 같아. 다음의 예시를 참고해서 이후에 내가 요청할 과제를 수행해줘. 예시에 대한 과제는 수행하지 않아도 돼.\r\n\r\n######################\r\n-예시-\r\n######################\r\n\r\n객체 유형 목록: [\"사람\", \"기업\"]\r\n\r\n뉴스 기사 제목: 네이버 이해진마저 엔비디아 젠슨 황 찾았다…AI 생태계 새 판 짜기\r\n\r\n뉴스 기사 작성일: 2024-06-28\r\n\r\n뉴스 기사 본문:\r\n\r\n생성형 인공지능(AI)에 투자하는 네이버가 'AI 반도체 1인자' 엔비디아와 협력을 진행하면서 AI 생태계 주도권 확보에 나섭니다.\r\n오늘(28일) 업계에 따르면 이해진 네이버 글로벌투자책임자(GIO)가 젠슨 황 엔비디아 최고경영자(CEO)를 만난 가운데 양사가 인공지능(AI) 소프트웨어∙하드웨어 조합의 파트너십을 구축할 전망입니다 지난 25일(현지시간) 이해진 GIO는 젠슨 황 엔비디아 CEO와 국가 고유의 역사, 문화를 반영한 소버린 AI 문제를 논의했습니다.\r\n이번 만남은 AI 소프트웨어에서 활로를 찾는 네이버가 AI 하드웨어 패권자인 엔비디아와 파트너십을 구축해 AI 주권을 뺏기지 않겠다는 의미로 풀이됩니다.\r\n소버린 AI는 정보 주권으로 국가나 기업이 자체적 인프라와 학습 데이터를 활용해 독립적인 AI 역량을 구축하는 것을 뜻합니다.\r\nAI가 학습하는 각 국가별 데이터에는 정치, 제도, 사회, 문화, 규율, 가치관이 반영됐습니다.\r\nAI 시대의 포문을 연 오픈AI의 챗GPT가 학습하는 데이터의 90%는 영어인 것으로 알려져 있습니다.\r\n이는 영어권 이외 국가의 정보는 왜곡된 내용으로 도출될 가능성이 커진다는 겁니다.\r\n하정우 네이버 퓨처 AI센터장은 지난 12일 오픈AI, 구글, 마이크로소프트가 개발한 자체 거대언어모델(LLM)과 관련해 \"한국어를 엄청나게 잘하지만 사고관은 뼛속까지 미국인인 AI\"라고 밝힌 바 있습니다.\r\n네이버는 영어와 중국어가 아닌 언어를 기반으로 초거대 AI 모델 구축의 원천기술을 보유하고 있습니다.\r\n이를 통해 클라우드 기반 AI 산업 생태계를 구축한 기업은 전 세계적으로 네이버가 유일합니다.\r\n소버린 AI 구축은 기업 간의 협의가 필수적이라고 여겨집니다.\r\n소버린 AI의 실현에 앞서 고성능 GPU(그래픽처리장치)를 보유한 데이터센터와 이를 구동할 수 있는 전력망, 데이터 수급을 위한 파이프라인과 생태계, 서비스를 갖춰야 하는 과제를 안고 있습니다.\r\n엔비디아는 고성능 GPU인 H100을 내세워 AI칩 시장을 장악한 가운데 글로벌 투자은행 시티그룹은 내년 엔비디아의 시장점유율이 약 81%가 될 것으로 예상했습니다.\r\n이 가운데 '탈엔비디아'를 선언한 네이버의 행보에는 엔비디아와의 접점 강화를 위한 포석으로 풀이됩니다. \r\n양사의 파트너십 구축으로 AI 하드웨어 패권자인 엔비디아 기술력에 네이버의 풍부한 데이터 자산이 합쳐지면 네이버는 한국에서 유일한 AI 기업이 될 것으로 해석됩니다.\r\n동시에 삼성전자와의 협업을 밝힌 네이버가 AI 가속기 ‘마하-1’을 개발 과정에서 삐걱거리면서 엔비디아가 새로운 '하드웨어' 파트너로 등극할지 관심이 모입니다. \r\n업계에선 향후 협업이 활발해질 경우 네이버와 엔비디아, TSMC, SK하이닉스 4사 협력 구도가 만들어질 가능성도 점치고 있습니다.\r\n\r\n################\r\n\r\n출력:\r\n\r\n- (\"객체\", \"네이버\", \"기업\", \"네이버는 생성형 인공지능(AI)에 투자하는 대한민국의 IT 기업이다.\")\r\n- (\"객체\", \"엔비디아\", \"기업\", \"엔비디아는 고성능 GPU인 H100을 내세워 AI칩 시장을 장악한 미국의 IT 기업이다.\")\r\n- (\"객체\", \"이해진\", \"사람\", \"이해진은 네이버의 글로벌투자책임자(GIO)이다.\")\r\n- (\"객체\", \"젠슨 황\", \"사람\", \"젠슨 황은 엔비디아의 최고경영자(CEO)이다.\")\r\n- (\"관계\", \"네이버\", \"엔비디아\", \"네이버와 엔비디아는 인공지능(AI) 소프트웨어∙하드웨어 조합의 파트너십을 구축했다.\", 5)\r\n- (\"관계\", \"이해진\", \"젠슨 황\", \"이해진과 젠슨 황은 국가 고유의 역사, 문화를 반영한 소버린 AI 문제를 논의했다.\", 3)\r\n- (\"관계\", \"네이버\", \"삼성전자\", \"네이버와 삼성전자는 협업을 통해 AI 가속기 ‘마하-1’개발 중이었으나 난항을 겪고 있다.\", -1)\r\n- (\"관계\", \"네이버\", \"TSMC\", \"네이버와 TSMC는 협력 구도로 인해 추후 협업 가능성이 존재한다.\", 2)\r\n- (\"관계\", \"네이버\", \"SK하이닉스\", \"네이버와 SK하이닉스는 협력 구도로 인해 향후 협업이 기대된다.\", 2)\r\n\r\n#############################"
print(instruction)

In [ ]:
sliding_window_executor = SlidingWindowExecutor(
    host='clovastudio.apigw.ntruss.com',
    api_key='NTA0MjU2MWZlZTcxNDJiY+Uy9rYumYZsV6izDa9AFvO56MDOrkyLi3eCmugWbNRu',
    api_key_primary_val = '1yiWJOyxczG1Ichq95JEUty5cZKr3F5iBynZfQWL',
    request_id='3dd95651-1c46-47e5-b56a-17b24d80669f'
)

completion_executor = ChatCompletionExecutor(
    host='https://clovastudio.stream.ntruss.com',
    api_key='NTA0MjU2MWZlZTcxNDJiY+Uy9rYumYZsV6izDa9AFvO56MDOrkyLi3eCmugWbNRu',
    api_key_primary_val='1yiWJOyxczG1Ichq95JEUty5cZKr3F5iBynZfQWL',
    request_id='c157480f-7a49-4fff-bc49-65227afaaffb'
)

In [ ]:
messages = [{"role": "user", "content": instruction}]

for index in news_info.keys():
    message_head = "객체 유형 목록: [\"기업\", \"인물\", \"국가\"]\n\n"
    news_title = "뉴스 기사 제목: " + news_info[index]['title'] + "\n\n"
    news_time = "뉴스 기사 작성일: " + news_info[index]['date'] + "\n\n"
    news_document = "뉴스 기사 본문: " + "\n\n" + news_info[index]['document'] 
    
    messages.append({"role": "user", "content": message_head + news_title + news_time + news_document})
    
    request_data = {
        "messages": messages,
        "maxTokens": 1024
    }

    try:
        adjusted_messages = sliding_window_executor.execute(request_data)
        if adjusted_messages == 'Error':
            print("Error adjusting messages with SlidingWindowExecutor")
            continue
    except Exception as e:
        print(f"Error adjusting messages: {e}")
        continue

    completion_request_data = {
        "messages": adjusted_messages,
        "maxTokens": 1024, 
        "temperature": 0.5,
        "topK": 0,
        "topP": 0.8,
        "repeatPenalty": 1.2,
        "stopBefore": [],
        "includeAiFilters": True,
        "seed": 0
    }

    try:
        response = completion_executor.execute(completion_request_data)
        extract_result = response.get('result', {}).get('message', {}).get('content', '').strip()
        print("NEWS {index}: \n\n", message_head + news_title + news_time + news_document)
        print("CLOVA Response: \n\n", extract_result, "\n")

    except Exception as e:
        print(f"Error: {e}")